# Redis Lists Tutorial

## Setup

In [1]:
%%capture
%pip install redis rich tqdm 

In [2]:
import redis
from rich.pretty import pprint

In [3]:
# creating cluster
# make sure jupyter server is connected to redis network
# `docker network connect redis_default jupyter-jupyter-1`
r = redis.RedisCluster(host='master', port=6379)

## Redis Lists

A Redis list is a data type that contains a series of ordered string values. Use cases include

- implementing stacks (LIFO data structures)
- implementing queues (FIFO data structures)
- implementing producer-consumer patterns

You interact with lists using commands such as:

- RPUSH - adds items to the end of a list.
- LPUSH - adds items to the beginning of a list.
- LLEN - retrieves the length of a list.
- LRANGE - retrieves list items from a specified range.
- LPOP - removes and returns an item from the beginning of a list.
- RPOP - removes and returns an item from the end of a list.

You can begin using with a list without first creating its key, simply by adding values to the key. This works as long as the key doesn't already exist as a different type.

**Note**:
> This is generally true for every Redis data structure, though there are a few exceptions.

If a key's values are removed entirely, the key will be removed from the keyspace.

In the following example, a list key is created and then a second element is added to it using the `RPUSH` command. The length of the list is returned after each command.

In [4]:
# Create a list with two elements
result1 = r.rpush("bike:colors", "Blue")
print(f"After first RPUSH, list length: {result1}")

result2 = r.rpush("bike:colors", "White")
print(f"After second RPUSH, list length: {result2}")

After first RPUSH, list length: 1
After second RPUSH, list length: 2


Next, an element is added to the beginning of the list.

In [5]:
# Prepend a new element
result = r.lpush("bike:colors", "Red")
print(f"After LPUSH, list length: {result}")
print(f"Current list: {r.lrange('bike:colors', 0, -1)}")

After LPUSH, list length: 3
Current list: [b'Red', b'Blue', b'White']


When creating a list, there's a shortcut to the above examples. Both `LPUSH` and `RPUSH` accept a variable number of arguments (variadic), so you can create the entire list with a single command. The number of added elements is returned.

In [6]:
# Add multiple elements
r.delete("bike:colors")
result = r.rpush("bike:colors", "Red", "Blue", "White", "Yellow")
print(f"Number of elements added: {result}")
print(f"Current list: {r.lrange('bike:colors', 0, -1)}")

Number of elements added: 4
Current list: [b'Red', b'Blue', b'White', b'Yellow']


The `LRANGE` command returns a subset of a key's elements. You provide two arguments in addition to the key name:

1. the index of the first element; note Redis lists use zero-based indexes
2. the index of the last element

For (2), a value of `-1` means the last element in the list; `-2` means the penultimate (second to last) element, and so on.

To return the entire list use `LRANGE keyname 0 -1`.

In [7]:
# LRANGE usage
print("Entire list:")
pprint(r.lrange("bike:colors", 0, -1))

print("\nElements at index 1 and 2:")
pprint(r.lrange("bike:colors", 1, 2))

print("\nElements from index 1 to second-to-last:")
pprint(r.lrange("bike:colors", 1, -2))

Entire list:


[b'Red', b'Blue', b'White', b'Yellow']


Elements at index 1 and 2:


[b'Blue', b'White']


Elements from index 1 to second-to-last:


[b'Blue', b'White']

To retrieve the number of elements in a list, use the `LLEN` command.

In [8]:
# LLEN usage
length = r.llen("bike:colors")
print(f"List length: {length}")

List length: 4


The `LPOP` and `RPOP` commands will remove and return one or more elements from the beginning or end of a list, respectively.

In this next example, you will remove a single element from the beginning of the list, then a single element from the end of the list, and, finally, the remaining items using a `count` argument.

In [10]:
# LPOP/RPOP usage
popped_element = r.lpop("bike:colors")
print(f"Element popped from the beginning: {popped_element}")
print(f"List after LPOP: {r.lrange('bike:colors', 0, -1)}")

popped_element = r.rpop("bike:colors")
print(f"\nElement popped from the end: {popped_element}")
print(f"List after RPOP: {r.lrange('bike:colors', 0, -1)}")

# In Python redis client, you can pop multiple elements by providing count parameter
popped_elements = r.lpop("bike:colors", 2)
print(f"\nElements popped with count=2: {popped_elements}")
print(f"List after LPOP with count=2: {r.lrange('bike:colors', 0, -1)}")

# Check if key still exists
existing_keys = r.keys("bike:colors")
print(f"\nDoes 'bike:colors' still exist? {'Yes' if existing_keys else 'No'}")

Element popped from the beginning: None
List after LPOP: []

Element popped from the end: None
List after RPOP: []

Elements popped with count=2: None
List after LPOP with count=2: []

Does 'bike:colors' still exist? No


**Note**:
> `LRANGE` will return an empty list if a key no longer exists.

### Resources

- List type [reference page](https://redis.io/docs/data-types/list?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
- Entire set of [Redis list commands](https://redis.io/commands/?group=list&utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
- Check out [Get started with Redis](https://university.redis.io/learningpath/14q8m6gilfwltm?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials) learning path on Redis University for an introduction to working with all core data structures in Redis.